In [9]:
import csv
import pandas as pd
import plotly.express as px

In [101]:
class Effect:
    all_effects = []
    bath = pd.read_pickle('data/bath_pickle.pkl')

    def __init__(self, description: str, lower_threshold: float, upper_threshold: float, cost_equation: str, units: str):
        self.description = description
        self.lower_threshold = lower_threshold
        self.upper_threshold = upper_threshold
        self.cost_equation = cost_equation
        self.units = units
        Effect.all_effects.append(self)

    def __repr__(self):
        return self.description

    def cost_function(self, elevation_m: float) -> float:
        if self.cost_equation == '':
            return 0
        MAX_SURFACE_AREA = 5574.65
        exposed_km2 = MAX_SURFACE_AREA - Effect.bath.at[round(elevation_m,2), 'Surface Area']
        return eval(self.cost_equation.split('=')[1])
    
    def filled_description(self, elevation: float) -> str:
        cost = self.format_cost_to_print(self.cost_function(elevation))
        return self.description.format(cost=cost)
    
    @staticmethod
    def format_cost_to_print(cost: float) -> str:
        if cost > 1000000000:
            word = 'billion'
            cost /= 1000000000
            return f'${cost:.1f} {word}'
        elif cost > 1000000:
            word = 'million'
            cost /= 1000000            
            return f'${cost:.1f} {word}'
        elif cost > 1000:
            return f'${round(cost/1000)},000'
        else:
            return f'${cost:.0f}'
    
    @classmethod
    def instantiate_from_csv(cls, path:str):
        with open(path, 'r') as f:
            reader = csv.DictReader(f)
            effects = list(reader)
        for effect in effects:
            if type(effect.get('effect_description')) is  None:
                print('nonetype')
            Effect(
                description = effect.get('effect_description'),
                lower_threshold = float(effect.get('lower_m')),
                upper_threshold = float(effect.get('upper_m')),
                cost_equation = effect.get('cost_equation'),
                units = effect.get('units')
            )

In [102]:
Effect.instantiate_from_csv('data/elevation_effects.csv')

print(Effect.all_effects[2].description)

The majority of wetlands are adversely affected.


In [2]:
class DataSchema:
    streamflow_after_consumption = 'streamflow_after_consumption'
    groundwater = 'groundwater'
    evap_km3_per_km2 = 'evap_km3_per_km2'
    percip_km3_per_km2 = 'percip_km3_per_km2'
    total_consumptive_use = 'total_consumptive_use'
    streamflow_before_consumption = 'streamflow_before_consumption'
    agriculture_consumption = 'agriculture_consumption'
    municipal_consumption = 'municipal_consumption'
    mineral_consumption =  'mineral_consumption'
    impounded_wetland_consumption = 'impounded_wetland_consumption'
    reservoir_consumption =  'reservoir_consumption'
    water_imports = 'water_imports'
    utah_population_millions = 'utah_population_millions'
    avg_elevation = 'avg_elevation'
[x for x in dir(DataSchema) if not x.startswith('__')]

['agriculture_consumption',
 'avg_elevation',
 'evap_km3_per_km2',
 'groundwater',
 'impounded_wetland_consumption',
 'mineral_consumption',
 'municipal_consumption',
 'percip_km3_per_km2',
 'reservoir_consumption',
 'streamflow_after_consumption',
 'streamflow_before_consumption',
 'total_consumptive_use',
 'utah_population_millions',
 'water_imports']

In [5]:
test = {'try': 2}
'2' in test

False

In [10]:
def CreateLineGraph(prediction: pd.DataFrame, lr_average_elevaton: float, df_lake: pd.DataFrame, units: str, rolling=5) -> px.scatter:

    MEAN_ELEVATION_BEFORE_1847 = 1282.3
    # trendline_y_points = [1281.7,1278.9]
    METERS_TO_FEET = 3.28084

    historic_avg_elevation = round(df_lake[DataSchema.avg_elevation].mean(),2)
    combined = pd.concat([df_lake,prediction],ignore_index=True)

    temp = pd.concat([combined.iloc[len(df_lake)-rolling:len(df_lake)][DataSchema.avg_elevation],combined.iloc[len(df_lake):]['Elevation Prediction']])
    combined['Elevation Prediction'] = temp

    if units == 'imperial':
        elevation_unit = 'ft'
        combined['Elevation Prediction'] *= METERS_TO_FEET
        lr_average_elevaton *= METERS_TO_FEET
        combined[DataSchema.avg_elevation] *= METERS_TO_FEET
        MEAN_ELEVATION_BEFORE_1847 *= METERS_TO_FEET
        historic_avg_elevation *= METERS_TO_FEET
        # trendline_y_points = [y * METERS_TO_FEET for y in trendline_y_points]
    else:
        elevation_unit = 'm'

    colors = ['blue','red']

    combined['datetime'] = pd.to_datetime(combined['datetime'])

    fig = px.scatter(combined, y=[DataSchema.avg_elevation,'Elevation Prediction'],
                        x='datetime', trendline='rolling',
                        trendline_options=dict(window=rolling),
                        color_discrete_sequence=colors,
                        labels = {
                            'value':f'Lake Elevation ({elevation_unit})',
                            'datetime':'Year'
                        },
                    )

    start_date = "1870-01-01"
    end_date = combined.at[len(combined)-1, "datetime"]
    fig.update_xaxes(type='date', range=[start_date, end_date])

    #only show trendlines
    fig.data = [t for t in fig.data if t.mode == 'lines']

    lr_pos = 'bottom left'
    human_pos = 'top left'

    fig.add_hline(y=MEAN_ELEVATION_BEFORE_1847, line_dash='dot',
                    annotation_text = f'Average Natural Level, {MEAN_ELEVATION_BEFORE_1847}{elevation_unit}',
                    annotation_position = 'top left',
                    annotation_font_size = 10,
                    annotation_font_color = 'black',
                    )

    fig.add_hline(y=historic_avg_elevation, line_dash='dot',
                annotation_text = f'Average Since 1847, {historic_avg_elevation}{elevation_unit}',
                annotation_position = human_pos,
                annotation_font_size = 10,
                annotation_font_color = colors[0],
                line_color = colors[0],
                )

    fig.add_hline(y=lr_average_elevaton, 
                    line_dash='dot',
                    annotation_text = f'Long-term Policy Average, {lr_average_elevaton}{elevation_unit}',
                    annotation_position = lr_pos,
                    annotation_font_size = 10,
                    annotation_font_color = colors[1],
                    line_color = colors[1],
            )

    return fig